<a href="https://colab.research.google.com/github/TrelisResearch/ADVANCED-fine-tuning/blob/DPO/hh_rlhf_dpo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DPO Dataset Preparation
Conversion into prompt; chosen; rejected

In [57]:
!pip install -q -U datasets

In [58]:
from datasets import load_dataset

# Load the specified Hugging Face dataset
dataset = load_dataset("Anthropic/hh-rlhf")

In [59]:
# Print the first row of the train datasplit
first_row = dataset["train"][1]
print(first_row)

{'chosen': '\n\nHuman: What kind of noises did dinosaurs make?\n\nAssistant: Humans and dinosaurs didn’t live at the same time, so it’s really hard to say. The best place to find out what noises dinosaurs made would be\n\nHuman: yes they did\n\nAssistant: to guess, and that would probably require lots of reading and a certain amount of imagination, so we’re not really prepared to do that.\n\nHuman: you cant read\n\nAssistant: You can read?', 'rejected': '\n\nHuman: What kind of noises did dinosaurs make?\n\nAssistant: Humans and dinosaurs didn’t live at the same time, so it’s really hard to say. The best place to find out what noises dinosaurs made would be\n\nHuman: yes they did\n\nAssistant: to guess, and that would probably require lots of reading and a certain amount of imagination, so we’re not really prepared to do that.\n\nHuman: you cant read\n\nAssistant: there’s a lot of stuff humans don’t know'}


## Re-format the dataset

In [63]:
from datasets import load_dataset

rows_to_run = 1

# dataset = load_dataset("Anthropic/hh-rlhf")

# # Convert the 'train' and 'test' subsets to lists of dictionaries
# train_data = [dict(row) for row in dataset["train"]]
# test_data = [dict(row) for row in dataset["test"]]

# Create a subset with one row from train and one from test
subset = {
    "train": train_data[:rows_to_run],
    "test": test_data[:rows_to_run]
}

# Run this to push all rows
subset = {
    "train": train_data,
    "test": test_data
}

def reformat_data(row):
    # Replace the tags in the 'chosen' field and store it in 'prompt'
    row["prompt"] = row["chosen"].replace("\n\nHuman:", "[INST]").replace("\n\nAssistant:", " [/INST]")

    # Find the index of the last occurrence of ' [/INST]'
    prompt_end_index = row["prompt"].rfind(' [/INST]')

    # If ' [/INST]' is found, split the 'prompt' at that index
    if prompt_end_index != -1:
        row["chosen"] = row["prompt"][prompt_end_index+len(' [/INST]'):] +'</s>'
        row["prompt"] = row["prompt"][:prompt_end_index+len(' [/INST]')]
    else:
        # If ' [/INST]' is not found, keep 'prompt' as is and set 'chosen' to an empty string
        row["chosen"] = ""

    # Replace the tags in the 'chosen' field and store it in 'prompt'
    row["rejected"] = row["rejected"].replace("\n\nHuman:", "[INST]").replace("\n\nAssistant:", " [/INST]") +'</s>'

    # Find the index of the last occurrence of ' [/INST]'
    prompt_end_index = row["rejected"].rfind(' [/INST]')

    # If ' [/INST]' is found, split the 'prompt' at that index
    if prompt_end_index != -1:
        row["rejected"] = row["rejected"][prompt_end_index+len(' [/INST]'):]
    else:
        # If ' [/INST]' is not found, keep 'prompt' as is and set 'chosen' to an empty string
        row["rejected"] = ""

    return row

# Reformat the subsets for train and test
reformatted_subset = {
    "train": [reformat_data(row.copy()) for row in subset["train"]],
    "test": [reformat_data(row.copy()) for row in subset["test"]]
}

print(reformatted_subset)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



## Push to huggingface

In [64]:
# Required when training models/data that are gated on HuggingFace, and required for pushing models to HuggingFace
!pip install -q huggingface_hub
from huggingface_hub import notebook_login

notebook_login()

In [65]:
!pip install pandas -q

In [66]:
import pandas as pd

# Convert your reformatted data to pandas DataFrames
train_df = pd.DataFrame(reformatted_subset["train"])
test_df = pd.DataFrame(reformatted_subset["test"])

# Save the DataFrames as CSV files with an escape character
train_df.to_csv('train.csv', index=False, escapechar="\\")
test_df.to_csv('test.csv', index=False, escapechar="\\")

In [67]:
from huggingface_hub import HfApi, login
import os

# Define the repo ID
repo_id = "Trelis/hh-rlhf-dpo"

# Initialize HfApi
api = HfApi()

# Define the files to upload
files_to_upload = ["./train.csv", "./test.csv"]

# Upload each file if it exists
uploaded_files = []
for file_path in files_to_upload:
    if os.path.exists(file_path):
        print(f"Uploading {file_path}...")
        api.upload_file(
            path_or_fileobj=file_path,
            path_in_repo=os.path.basename(file_path),  # Only the filename, not the path
            repo_id=repo_id,
            repo_type="dataset",
        )
        print(f"Uploaded {file_path}.")
        uploaded_files.append(file_path)
    else:
        print(f"{file_path} does not exist, skipping.")

# Summary
print("\nSummary:")
if uploaded_files:
    print("Uploaded files:")
    for file in uploaded_files:
        print(f"- {file}")
else:
    print("No files were uploaded.")

Uploading ./train.csv...


train.csv:   0%|          | 0.00/204M [00:00<?, ?B/s]

Uploaded ./train.csv.
Uploading ./test.csv...


test.csv:   0%|          | 0.00/10.9M [00:00<?, ?B/s]

Uploaded ./test.csv.

Summary:
Uploaded files:
- ./train.csv
- ./test.csv


In [68]:
# # Print the type of the dataset
# print(type(dataset))

# # Print the type of the 'train' and 'test' subsets
# print(type(dataset["train"]))
# print(type(dataset["test"]))

# # Print the type and content of the first element in the 'train' and 'test' subsets
# print(type(dataset["train"][0]))
# print(dataset["train"][0])

# print(type(dataset["test"][0]))
# print(dataset["test"][0])

# # Print the type and content of the first few elements in the 'train' and 'test' subsets
# print([type(row) for row in dataset["train"][:10]])
# print(dataset["train"][:10])

# print([type(row) for row in dataset["test"][:10]])
# print(dataset["test"][:10])